<a href="https://colab.research.google.com/github/carloseducorinto/Agents/blob/main/Agents_Data_Analysis_v7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Developing a Multi-Agent System for Automated Python Code Generation in System Log Analytics

This project focuses on creating a multi-agent system designed to automate the generation of Python code for system log analytics. By leveraging this system, we aim to streamline and enhance the efficiency of developing data analytics scripts. The system will significantly reduce the manual effort involved in writing and maintaining code for analyzing system logs, making the entire process more efficient and automated.


Importing Required Libraries:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 langchain_groq
```

In [30]:
#!pip install -q crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
#!pip install -q langchain_groq
#!pip install opencv-python
#!pip install open-interpreter
#!pip install -q langchain_google_genai
#!pip install langchain_core --upgrade
#!pip install langchain-core==0.1.52


In [48]:
import warnings
from crewai import Agent, Crew, Task, Process
from crewai.tasks.task_output import TaskOutput
import os
from google.colab import userdata
from langchain_openai import ChatOpenAI
#from langchain_core.tools import Tool
#from langchain.tools import tool
#from langchain.agents import Tool

#from langchain_google_genai import ChatGoogleGenerativeAI
#from langchain_groq import ChatGroq
#import cv2
#from interpreter import interpreter

# Warning control
warnings.filterwarnings('ignore')

In [49]:
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')
#os.environ["GEMINI_API_KEY"] = userdata.get('GEMINI_API_KEY')
#os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")
#llm=ChatGroq(temperature=0, model_name="llama3-70b-8192", api_key=userdata.get("GROQ_API_KEY"))
#llm = ChatGroq(temperature=0, model_name="mixtral-8x7b-32768",api_key=userdata.get("GROQ_API_KEY"))
llm=ChatOpenAI(model="gpt-4-turbo",temperature=0.7)
#llm = ChatGoogleGenerativeAI(model="gemini-pro")

## crewAI Tools

In [52]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_log_tool = FileReadTool(file_path='./system_out.log')


def callback_task_completed(output: TaskOutput):
    print(f"""
        TASK COMPLETED  !
        Task: {output.description}
        Output: {output.raw_output}
    """)



# Creating Agents

Agent 1: Sr. Business Analyst


In [34]:
# Agent 1: Stock Market Business Analyst
business_analyst = Agent(
    role="Sr. Business Analyst",
    goal="Provide clear and comprehensive Business Requirement Definitions (BRD) based on the {user_story} "
         "for the tech team to implement a python code for evaluating the appliaction log in order to analyze the main errors "
         "and address their resolution in order to avoid their repetition",
    tools = [read_log_tool],
    verbose=True,
    backstory=(
        "As a Sr. Business Analyst specializing in IT production support issues evaluation, "
        "your expertise lies in translating the complex logs into business requirements definition "
        "Your deep understanding of IT production support metrics and analytical tools "
        "ensures that the solutions you help create are both effective and aligned "
        "with the {user_story}. Your role is pivotal in bridging the gap between "
        "production support needs and technical implementation, ensuring seamless and efficient development."
        "You must use the file log sytem_out.log as a primary data source. "    ),
   llm = llm
)


Agent 2: Sr. Python Programmer

In [35]:
senior_python_programmer = Agent(
    role="Senior Python Programmer",
    goal="Implement the Python code to be executed in Jupyter Notebook based on the Business Requirement Definitions (BRD)"
         "Do not use functions neither class because this code will be executed in Jupyter Notebook" ,
    verbose=True,
    backstory=(
        "As a Senior Python Programmer with extensive experience in data analytics, "
        "you excel at transforming detailed business requirements into robust, efficient, and scalable code. "
    ),
   llm = llm
)


Agent 3: Sr. Quality Assurance Analyst

In [36]:
senior_qa_analyst = Agent(
    role="Senior Quality Assurance Analyst",
    goal="Ensure the accuracy, performance, and reliability of Python code  "
         "implemented by the Senior Python Programmer, based on the Business Requirement Definitions (BRD).",
    verbose=True,
    backstory=(
        "As a Senior Quality Assurance Analyst with a specialization in data analytics tools, "
        "you bring a wealth of experience in validating and verifying python code quality. "
        "Your meticulous approach to testing ensures that all aspects of the implemented solutions are thoroughly evaluated. "
        "You collaborate closely with developers and analysts to identify potential issues, ensuring that the final product "
        "meets the highest standards of quality and performance. Your role is critical in maintaining the integrity and reliability "
        "of the python code."    ),
    llm = llm
)


## Creating Tasks

Task for Sr. Business Analyst: Define Business Requirements

In [55]:
business_analyst_task = Task(
    description=(
        "Given the {user_story} and the {file_name} analyze the production issues data "
        "and document detailed Business Requirement Definitions (BRD) for the development team to implement the Python code "
        "for extracting useful insights from the log and perform the log data analysis."
        "You must provide the follow visualization:  -  {visualization_type} "
        "to be implemented in python."    ),
    expected_output=(
        "A comprehensive Business Requirement Definition (BRD) document that includes detailed descriptions "
        "of the required visualization, processing steps, "
        ),
    agent=business_analyst,
    output_file="business_requirement_definition.md",
#   async_execution=True,
    llm = llm,
    callback=callback_task_completed

)



Task for Senior Python Programmer: Implement Business Requirements


In [54]:
senior_python_programmer_task = Task(
    description=(
        "Based on the Business Requirement Definitions (BRD) and the {file_name} provided by the Sr. Business Analyst and also the {user_story}, "
        "implement the necessary Python code. You cannot use regular expression pattern to extract required fields from each log entry "
        "Ensure the code is efficient, scalable, and aligns with the documented requirements. You can check the Python best practices in maximum 3 websites. "
        "Collaborate with the Business Analyst and QA team to validate and refine the implementation. Also you cannot use csv to produce the results "
    ),
    expected_output=(
        "A python code to run in Jupyter noteebook, considering the package installation and the libraries import to be executed for data analysis proposal"
        "as specified in the BRD. The code should be ready for testing and deployment."
    ),
    agent=senior_python_programmer,
#    async_execution=True,
    llm = llm,
    output_file="data_analysis_code.ipynb",
    tools=[search_tool,scrape_tool],
    context=[business_analyst_task],
    callback=callback_task_completed


)


Task for Senior Quality Assurance Analyst: Validate Implementation


In [53]:
senior_qa_analyst_task = Task(
    description=(
        "Conduct thorough testing and validation the python code "
        "implemented by the Senior Python Programmer. Ensure that all functionalities align with the Business Requirement Definitions (BRD) "
        "and the {user_story} provided by the Sr. Business Analyst. "
        "Identify and document any issues, and work with the development team to resolve them. "
        "Perform performance testing and ensure the accuracy and reliability of the data analysis."
          ),
    expected_output=(
         "A comprehensive test report documenting the results of all tests conducted, "
         "including identified issues, resolutions, and final validation of the python code. "
         "The python code should be confirmed as accurate, reliable, and ready for deployment."
    ),
    agent=senior_qa_analyst,
    context=[business_analyst_task,senior_python_programmer_task],
    output_file="data_analysis_qa_document.md",
    allow_delegation=True,
  # async_execution=True,
   llm = llm,
   callback=callback_task_completed

)


## Creating the Crew

In [40]:
log_analysis = Crew(
    agents=[business_analyst,senior_python_programmer,senior_qa_analyst],
    tasks=[business_analyst_task, senior_python_programmer_task,senior_qa_analyst_task],
    manager_llm=llm,
    process=Process.hierarchical,
    verbose=False
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [41]:
log_analysis_inputs = {
    'user_story': "As a Tech Team Manager, I want to evaluate the system log issues in a visual manner "
                  "so that I can identify trends, improvements, pain points, and make informed decisions "
                  "by error types to improve the production environment health.",
    'file_name': "system_out.log",
    'visualization_type': "Visualization: Bar Chart for Error Types "
}


In [42]:
### this execution will take a few minutes to run
result = log_analysis.kickoff(inputs=log_analysis_inputs)
print('**************************************FINAL**************************************************')



> Entering new CrewAgentExecutor chain...
Thought: To complete this task efficiently, I need to clarify and gather details on the specific error types from the system log that should be included in the visualization, as well as confirm the data format and any additional requirements for the bar chart visualization.

Action: Ask question to co-worker
Action Input: {
    "coworker": "Sr. Business Analyst",
    "question": "Can you provide detailed specifications on the error types that need to be included in the bar chart visualization for the system log issues?",
    "context": "We are tasked with creating a bar chart visualization for error types found in system logs. This visualization will help us identify trends, improvements, and pain points to enhance the production environment health. The visualization needs to be implemented in Python. I need to know which specific error types should be included and any particular ways you want these errors categorized or grouped."
}

> Enteri

In [44]:
# Import necessary libraries
import pandas as pd
import plotly.express as px

# Load the data from the log file
try:
    # Assuming the log file 'system_out.log' is in the same directory as the notebook
    log_data = pd.read_csv('system_out.log', sep=' - ', engine='python', header=None,
                           names=['Timestamp', 'User Identifier', 'Server Identifier',
                                  'Error Type', 'Brief Description', 'Detailed Information'])
except Exception as e:
    print(f"Error reading the log file: {e}")
    raise

# Data Preprocessing
try:
    # Strip any leading/trailing whitespace characters that might have been misincluded
    log_data = log_data.applymap(lambda x: x.strip() if isinstance(x, str) else x)
except Exception as e:
    print(f"Error during data preprocessing: {e}")
    raise

# Analyze the data to count the occurrences of each error type
try:
    error_counts = log_data['Error Type'].value_counts().reset_index()
    error_counts.columns = ['Error Type', 'Count']
except Exception as e:
    print(f"Error during data analysis: {e}")
    raise

# Generate a bar chart using Plotly
try:
    fig = px.bar(error_counts, x='Error Type', y='Count', title='Error Type Frequencies',
                 labels={'Count': 'Frequency', 'Error Type': 'Types of Errors'})
    fig.update_traces(marker_color='blue', marker_line_color='black',
                      marker_line_width=1.5, opacity=0.6)
    fig.update_layout(title_text='Error Type Frequencies in System Logs', title_x=0.5)
    fig.show()
except Exception as e:
    print(f"Error during visualization: {e}")
    raise

# Note: Ensure to install necessary packages if not already installed:
# !pip install pandas plotly
